In [4]:
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from pycountry import countries
import numpy as np
from plotly.subplots import make_subplots
import statsmodels.api as sm
import pingouin as pg


In [5]:
# Emissions data is CO2 emissions, traffic the total passengers, Tourism the Nights spent at tourist accommodation establishments, 
# and aviation the total freight and mail exported monthly

emissions = pd.read_csv('Datasets/emissions.csv')
traffic = pd.read_csv('Datasets/traffic.csv')
tourism = pd.read_csv('Datasets/tourism.csv')
supply = pd.read_csv('Datasets/aviation_freight.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/emissions.csv'

### Schiphol, The Netherlands

In [14]:
traffic_NL = traffic[
    (traffic['rep_airp'] == 'NL_EHAM') &
    (traffic['unit'] == 'NR') &
    (traffic['TIME_PERIOD'] >= '2019-02')                   
]

emissions_NL = emissions[
    (emissions['geo'] == 'NL_CAP') &
    (emissions['TIME_PERIOD'] >= '2019-02')
]

tourism_NL = tourism[
    (tourism['geo'] == 'NL') &
    (tourism['TIME_PERIOD'] >= '2019-02')
]

supply_NL = supply[
    (supply['geo'] == 'NL') &
    (supply['TIME_PERIOD'] >= '2019-02')
]

In [1]:
tourism_NL.unit

NameError: name 'tourism_NL' is not defined

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a 3x1 subplot layout
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    subplot_titles=("Traffic vs Emissions", "Traffic vs Tourism", "Traffic vs Freight"),
                    vertical_spacing=0.1)

# For Traffic vs Emissions
fig.add_trace(go.Scatter(
    x=traffic_NL['TIME_PERIOD'],
    y=traffic_NL['OBS_VALUE'],
    name="Traffic [Number]",
    line=dict(color='blue'),
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=emissions_NL['TIME_PERIOD'],
    y=emissions_NL['OBS_VALUE'],
    name="Emissions (NO_2)",
    line=dict(color='green'),
    yaxis="y2"
), row=1, col=1)

# For Traffic vs Tourism
fig.add_trace(go.Scatter(
    x=traffic_NL['TIME_PERIOD'],
    y=traffic_NL['OBS_VALUE'],
    showlegend=False,  # Hide legend as it's the same traffic data
    line=dict(color='blue'),
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=tourism_NL['TIME_PERIOD'],
    y=tourism_NL['OBS_VALUE'],
    name="Tourism",
    line=dict(color='red'),
    yaxis="y3"
), row=2, col=1)

# For Traffic vs Freight
fig.add_trace(go.Scatter(
    x=traffic_NL['TIME_PERIOD'],
    y=traffic_NL['OBS_VALUE'],
    showlegend=False,  # Hide legend as it's the same traffic data
    line=dict(color='blue'),
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=supply_NL['TIME_PERIOD'],
    y=supply_NL['OBS_VALUE'],
    name="Freight NL",
    line=dict(color='black'),
    yaxis="y4"
), row=3, col=1)

# Update layout for better appearance
fig.update_layout(
    title_text="Impact of COVID-19 on Different Sectors in The Netherlands",
    height=900  # You can adjust this for the desired height
)

fig.show()


### France


### Spain

In [21]:
# Create figure
fig = go.Figure()

# Add traces for traffic
fig.add_trace(go.Scatter(
    x=traffic_NL['TIME_PERIOD'],
    y=traffic_NL['OBS_VALUE'],
    name="Traffic [Number]",
    line=dict(color='blue'),
))

# Add traces for emissions
fig.add_trace(go.Scatter(
    x=emissions_NL['TIME_PERIOD'],
    y=emissions_NL['OBS_VALUE'],
    name="Emissions (NO_2)",
    line=dict(color='green'),
    yaxis="y2"
))

# Add a third trace for traffic change in percentage
fig.add_trace(go.Scatter(
    x=traffic_NL['TIME_PERIOD'],
    y=traffic_NL['OBS_VALUE'],
    name="Traffic Change [%]",
    line=dict(color='blue', dash = 'dash' ),
    yaxis="y3"  # Designating the third y-axis for this trace
))

# Update layout for tri-axis
fig.update_layout(
    title='Traffic, Emissions, and Traffic Change Over Time in Amsterdam',
    xaxis=dict(title='TIME_PERIOD'),
    yaxis=dict(title='Traffic [Number]'),
    yaxis2=dict(
        title='Emissions (NO_2)',
        overlaying='y',
        side='right'
    ),
    yaxis3=dict(
        title='Traffic Change [%]',
        overlaying='y',
        side='left',  # You may want to set this to 'right' if overlay becomes a concern
        anchor="free",
        position=0.05  # The position is a fraction of the plot width
    )
)

fig.add_hline(y=24000, line_color = 'red', line_dash = 'dot')
# Show the figure
fig.show()